In [2]:
import pandas as pd
import os
import numpy as np 
import joblib as joblib

In [89]:
# dataFiles = {dataFile :["./data/Parts_EQP_Output_ByMonth_20210407_van.csv"
#                 ,"./data/ScmTrainingData_Monthly_30days.csv"
#                 ,"./data/holiday.csv"
#                 ],
#             relation :[['MFG_MONTH','EQP_NO'],['MFG_MONTH','TOOL_ID']]
#                 ,[['MFG_MONTH'],['MFG_MONTH']]
#                ]

In [58]:
# datasetList =datafileList[0:1][0]

# datasetRels =datafileList[1:][0]
# datasetRels[0]

['MFG_MONTH', 'EQP_NO']

In [79]:
def readData(inputfile):
        df = pd.read_csv(inputfile)
        df=df.dropna(axis=1,how='all')
        df.info() 
        return analyzeData(df) 
def analyzeData(df):
        print('非數值欄位：')
        strColumnlist=df.select_dtypes(exclude=['int64','float64']).columns.tolist()
        print(strColumnlist) 
        print('數值欄位：')
        numbericColumnlist=df.select_dtypes(include=['int64','float64']).columns.tolist()
        print(numbericColumnlist)
        print('包含ＮＵＬＬ的欄位：')
        nullColumnlist=df.columns[df.isna().any()].tolist()    
        print(nullColumnlist)
        print('===================================================')  
        return df,strColumnlist,numbericColumnlist,nullColumnlist        
def mergeDataFrame(dfleft,dfright,LeftKeys,RightKeys):
        # dfright.columns = [str(col) + '_'+joinTableName for col in df.columns]
        df_merge = pd.merge(dfleft, dfright, left_on=LeftKeys, right_on=RightKeys,how="inner")
        return df_merge        

In [100]:
dataFiles = {'files':["./data/Parts_EQP_Output_ByMonth_20210407_van.csv"
                ,"./data/ScmTrainingData_Monthly_30days.csv"
                ,"./data/holiday.csv"
                ],
            'relations':[
                 [['MFG_MONTH','EQP_NO'],['MFG_MONTH','TOOL_ID']]
                ,[['MFG_MONTH'],['MFG_MONTH']]
               ]
}

In [103]:
dataFiles['files']

['./data/Parts_EQP_Output_ByMonth_20210407_van.csv',
 './data/ScmTrainingData_Monthly_30days.csv',
 './data/holiday.csv']

In [106]:


def merge(dataFiles):
    index =0
    for dfFile in dataFiles['files']:
        print(dfFile)
        if index ==0:
            _dfInputData1,_strColumnlist1,_numbericColumnlist1,_nullColumnlist1=readData(dfFile) 
            
            # _df_result.
        else:
            datasetRels = dataFiles['relations'][index-1]
            _dfInputData2,_strColumnlist2,_numbericColumnlist2,_nullColumnlist2=readData(dfFile) 
            _dfInputData1.set_index(datasetRels[0])
            _dfInputData2.set_index(datasetRels[1])
            df_merge = mergeDataFrame(_dfInputData1,_dfInputData2,datasetRels[0],datasetRels[1])
            _dfInputData1 = df_merge.copy(deep=False)
        index+=1
    return _dfInputData1 
df = merge(dataFiles)

./data/Parts_EQP_Output_ByMonth_20210407_van.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20775 entries, 0 to 20774
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   PART_NO           20775 non-null  object 
 1   EQP_NO            20775 non-null  object 
 2   STOCK_EVENT_TIME  20775 non-null  object 
 3   QTY               20775 non-null  int64  
 4   PM                20775 non-null  float64
 5   TS                20775 non-null  float64
 6   ENG               20775 non-null  float64
 7   NST               20775 non-null  float64
 8   MFG_MONTH         20775 non-null  int64  
dtypes: float64(4), int64(2), object(3)
memory usage: 1.4+ MB
非數值欄位：
['PART_NO', 'EQP_NO', 'STOCK_EVENT_TIME']
數值欄位：
['QTY', 'PM', 'TS', 'ENG', 'NST', 'MFG_MONTH']
包含ＮＵＬＬ的欄位：
[]
./data/ScmTrainingData_Monthly_30days.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1837 entries, 0 to 1836
Data columns (total 26 columns):

In [108]:
# _dfInputData1.reset_index(drop=True, inplace=True)
# _dfInputData1.drop(['index'],axis=1)
df.columns


Index(['PART_NO', 'EQP_NO', 'STOCK_EVENT_TIME', 'QTY', 'PM', 'TS', 'ENG',
       'NST', 'MFG_MONTH', 'TOOL_ID', 'M_NUM', 'UP_TIME', 'C_UP_TIME',
       'LOT_SIZE', 'C_LOT_SIZE', 'EQP_UTIL', 'C_EQP_UTIL', 'EQP_AVAIL_RATE',
       'TC', 'PROCESS_TIME', 'PROCESS_JOBTIME', 'INLINE_CT_LOT',
       'INLINE_CT_WAFER', 'MOVE_QTY', 'C_TC', 'ENG_LOT_RATE', 'HOT_LOT_RATE',
       'BACKUP_BY_RATE', 'BACKUP_FOR_RATE', 'RWORK_LOT_RATE', 'QUE_LOT_RATE',
       'SAMPLING_RATE', 'NUM_RECIPE', 'CHANGE_RECIPE', 'Holiday', 'Days'],
      dtype='object')

In [65]:
# df_parts=pd.read_csv("./data/Parts_EQP_Output_ByMonth_20210407_2.csv")
# df_parts['MFG_MONTH'] = df_parts['STOCK_EVENT_TIME'].apply(lambda x: pd.to_datetime(str(x), format='%Y/%m/%d %H:%M')).dt.strftime('%Y%m')
# df_parts.to_csv("./data/Parts_EQP_Output_ByMonth_20210407_van.csv",index=False)

In [62]:
df_parts=pd.read_csv("./data/Parts_EQP_Output_ByMonth_20210407_van.csv")
df_parts['MFG_MONTH'] = pd.to_datetime(df_parts['STOCK_EVENT_TIME'].values, format='%Y-%m-%d').astype('period[Q]')
df_parts.groupby(['PART_NO','EQP_NO','MFG_MONTH']).sum().reset_index()
df_EQP=pd.read_csv("./data/ScmTrainingData_Monthly_30_20152021.csv")
df_EQP['MFG_MONTH'] = pd.to_datetime(df_EQP['MFG_MONTH'].values, format='%Y%m').astype('period[Q]')
df_EQP.groupby(['TOOL_ID','MFG_MONTH']).mean().reset_index()
pd.merge(df_parts, df_EQP, left_on=['EQP_NO','MFG_MONTH'], right_on=['TOOL_ID','MFG_MONTH'],how="inner")

,TOOL_ID,MFG_MONTH,M_NUM,UP_TIME,C_UP_TIME,LOT_SIZE,C_LOT_SIZE,EQP_UTIL,C_EQP_UTIL,EQP_AVAIL_RATE,...,C_TC,ENG_LOT_RATE,HOT_LOT_RATE,BACKUP_BY_RATE,BACKUP_FOR_RATE,RWORK_LOT_RATE,QUE_LOT_RATE,SAMPLING_RATE,NUM_RECIPE,CHANGE_RECIPE
0,DIA01,2015Q1,3.0,0.913123,0.046450,22.166542,0.293950,0.911978,0.234825,3.560113,...,0.509202,0.003077,0.024012,0.0,0.071718,0.000102,0.928180,0.0,21.000000,0.649773
1,DIA01,2015Q2,3.0,0.843459,0.050789,21.387001,0.461008,0.839925,0.266091,3.067737,...,0.542214,0.003140,0.033550,0.0,0.000000,0.002430,0.997156,0.0,44.000000,0.815174
2,DIA01,2015Q3,3.0,0.873781,0.051042,21.927232,0.365429,0.924559,0.238711,3.520805,...,0.555230,0.004928,0.013528,0.0,0.000000,0.000382,0.999618,0.0,31.333333,0.574232
3,DIA01,2015Q4,3.0,0.877797,0.053552,22.479113,0.306193,0.924369,0.234177,3.671709,...,0.501822,0.006104,0.023428,0.0,0.052450,0.001112,0.946869,0.0,22.333333,0.594151
4,DIA01,2016Q1,3.0,0.913123,0.046450,22.166542,0.293950,0.911978,0.234825,3.560113,...,0.509202,0.003077,0.024012,0.0,0.071718,0.000102,0.928180,0.0,21.000000,0.649773
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3307,WVT02,2020Q1,1.0,0.970339,0.041758,23.003453,0.477843,0.990959,0.222740,5.358815,...,0.450503,0.014378,0.061748,0.0,0.028827,0.001472,0.936525,0.0,22.500000,0.846142
3308,WVT02,2020Q2,1.0,0.963842,0.044825,23.192654,0.393632,0.986904,0.225712,5.061473,...,0.455797,0.005768,0.047092,0.0,0.033837,0.002145,0.949200,0.0,30.666667,0.880870
3309,WVT02,2020Q3,1.0,0.965723,0.047085,22.749617,0.332871,0.983728,0.222388,5.712435,...,0.461069,0.012419,0.062802,0.0,0.032061,0.000671,0.943782,0.0,21.000000,0.863158
3310,WVT02,2020Q4,1.0,0.967670,0.048026,23.045571,0.286807,0.980721,0.220027,5.827443,...,0.448182,0.009567,0.042770,0.0,0.056707,0.001686,0.927821,0.0,15.000000,0.861022


In [65]:
pd.merge(df_parts, df_EQP, left_on=['EQP_NO','MFG_MONTH'], right_on=['TOOL_ID','MFG_MONTH'],how="inner")

,PART_NO,EQP_NO,STOCK_EVENT_TIME,QTY,PM,TS,ENG,NST,MFG_MONTH,TOOL_ID,...,C_TC,ENG_LOT_RATE,HOT_LOT_RATE,BACKUP_BY_RATE,BACKUP_FOR_RATE,RWORK_LOT_RATE,QUE_LOT_RATE,SAMPLING_RATE,NUM_RECIPE,CHANGE_RECIPE
0,85-ECT0010,ECT11,2015-01-31,0,0.0,0.0,0.0,0.0,2015Q1,ECT11,...,0.416924,0.033649,0.027876,0,0.000000,0.002722,0.932041,0,39,0.900391
1,85-ECT0010,ECT11,2015-01-31,0,0.0,0.0,0.0,0.0,2015Q1,ECT11,...,0.439473,0.020306,0.047381,0,0.000000,0.009491,0.869188,0,59,0.906195
2,85-ECT0010,ECT11,2015-01-31,0,0.0,0.0,0.0,0.0,2015Q1,ECT11,...,0.476868,0.020802,0.088825,0,0.166262,0.004631,0.717051,0,48,0.867857
3,85-ECT0010,ECT11,2015-02-28,0,0.0,0.0,0.0,0.0,2015Q1,ECT11,...,0.416924,0.033649,0.027876,0,0.000000,0.002722,0.932041,0,39,0.900391
4,85-ECT0010,ECT11,2015-02-28,0,0.0,0.0,0.0,0.0,2015Q1,ECT11,...,0.439473,0.020306,0.047381,0,0.000000,0.009491,0.869188,0,59,0.906195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60343,85-EMA0920,EWA52,2021-02-28,0,0.0,0.0,0.0,0.0,2021Q1,EWA52,...,0.435650,0.001397,0.007784,0,0.152096,0.000499,0.699202,0,41,0.853832
60344,85-EMA0920,EWA52,2021-02-28,0,0.0,0.0,0.0,0.0,2021Q1,EWA52,...,0.417824,0.004367,0.013974,0,0.074585,0.013800,0.768035,0,52,0.866771
60345,85-EMA0920,EWA52,2021-03-31,1,1.0,0.0,0.0,0.0,2021Q1,EWA52,...,0.404703,0.011236,0.007745,0,0.045271,0.003491,0.824588,0,36,0.873047
60346,85-EMA0920,EWA52,2021-03-31,1,1.0,0.0,0.0,0.0,2021Q1,EWA52,...,0.435650,0.001397,0.007784,0,0.152096,0.000499,0.699202,0,41,0.853832


In [64]:
df_parts

,PART_NO,EQP_NO,STOCK_EVENT_TIME,QTY,PM,TS,ENG,NST,MFG_MONTH
0,85-ECT0010,ECT11,2015-01-31,0,0.0,0.0,0.0,0.0,2015Q1
1,85-ECT0010,ECT11,2015-02-28,0,0.0,0.0,0.0,0.0,2015Q1
2,85-ECT0010,ECT11,2015-03-31,0,0.0,0.0,0.0,0.0,2015Q1
3,85-ECT0010,ECT11,2015-04-30,0,0.0,0.0,0.0,0.0,2015Q2
4,85-ECT0010,ECT11,2015-05-31,0,0.0,0.0,0.0,0.0,2015Q2
...,...,...,...,...,...,...,...,...,...
20770,85-EMA0920,EWA53,2020-11-30,0,0.0,0.0,0.0,0.0,2020Q4
20771,85-EMA0920,EWA53,2020-12-31,0,0.0,0.0,0.0,0.0,2020Q4
20772,85-EMA0920,EWA53,2021-01-31,0,0.0,0.0,0.0,0.0,2021Q1
20773,85-EMA0920,EWA53,2021-02-28,0,0.0,0.0,0.0,0.0,2021Q1


In [2]:
partList =['85-ECT0010','85-EKA0190','85-EKA0270','85-EMA0130','85-EMA0900','85-EMA0910','85-EMA0920','86-DIA0120','87-WPT1070']
for p in partList:
    print(p)

85-ECT0010
85-EKA0190
85-EKA0270
85-EMA0130
85-EMA0900
85-EMA0910
85-EMA0920
86-DIA0120
87-WPT1070
